# Spark launcher

In [1]:
# !conda create --name old_spark  python=3.7 -y

In [2]:
# DIR="$(cd -- "$(dirname -- "${0}")" && pwd)"
# export ROOT_DIR="$(cd -- "$(dirname -- "${0}")" && cd .. && pwd)"
# export SERP_HOME="${SERP_HOME:-$ROOT_DIR/env/serp}"
# export PATH="$SERP_HOME/bin:$ROOT_DIR/env/ds/bin:$ROOT_DIR/env/ds/opt/anaconda/bin:$PATH"


In [8]:
# !export PIP_INDEX_URL=https://atanona:AKCp8k8Prres8xHvKmJiNbR8APWLKavCZbUox2zBqXNwYF4wozziyjrpB3rgkP6SCPwqvgxtg@repo.silenteight.com/artifactory/api/pypi/pypi/simple; pip install -r requirements.txt

In [9]:
# !export PIP_INDEX_URL=https://atanona:AKCp8k8Prres8xHvKmJiNbR8APWLKavCZbUox2zBqXNwYF4wozziyjrpB3rgkP6SCPwqvgxtg@repo.silenteight.com/artifactory/api/pypi/pypi/simple; pip install -r python-datatoolbox/pip-requirements.txt

In [6]:
# !pip install pyspark==3.1.1

# Create standardized delta/csv files - 1.0-raw-to-standardized - DeltaConverter, Spark Manager

In [24]:
from silenteight.utils.jupyter import set_jupyter_cell_width, set_pd_display
set_jupyter_cell_width()
set_pd_display()

/env/ds/anaconda/envs/old_spark/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/env/ds/anaconda/envs/old_spark/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/env/ds/anaconda/envs/old_spark/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Convert raw data to standardized format

In [27]:
from pipeline.preprocessors import convert_to_standardized, spark_instance, RAW_DATA_DIR, STANDARDIZED_DATA_DIR

In [28]:
convert_to_standardized(RAW_DATA_DIR, STANDARDIZED_DATA_DIR)

2021/12/09 17:18:49 - root INFO: Start to process ./data/1.raw/ACM_ALERT_NOTES.csv


./data/1.raw/
./data/1.raw/


2021/12/09 17:18:51 - root INFO: Data saved to ./data/2.standardized/ACM_ALERT_NOTES.delta
2021/12/09 17:18:51 - root INFO: Time lapsed 1.79 s


2021/12/09 17:18:52 - root INFO: Start to process ./data/1.raw/ACM_MD_ALERT_STATUSES.csv


./data/1.raw/
./data/1.raw/


...

./data/1.raw/
./data/1.raw/


2021/12/09 17:18:55 - root INFO: Data saved to ./data/2.standardized/ACM_ITEM_STATUS_HISTORY.delta
2021/12/09 17:18:55 - root INFO: Time lapsed 1.13 s


2021/12/09 17:18:56 - root INFO: Start to process ./data/1.raw/ALERTS.csv


./data/1.raw/
./data/1.raw/


2021/12/09 17:18:57 - root INFO: Data saved to ./data/2.standardized/ALERTS.delta
2021/12/09 17:18:57 - root INFO: Time lapsed 0.68 s


# Create cleansed csv files - 2.0-standardized-to-cleansed - Spark manager, XML Parser, Match/Hit Handler,Delta Converter, NRIC handler, Status Preprocessor, Notes Preprocessor

In [21]:
from silenteight.aia.alerts import AlertHitDictFactory, AlertHitExtractor
from pyspark.sql.types import StructType, ArrayType, StructField
import pyspark.sql.functions as F

# Implementation: XML Parser
alert_hit_dict_factory = AlertHitDictFactory()
alert_hit_extractor = AlertHitExtractor()

# Implementation: XML Parser
alert_schema = alert_hit_dict_factory.get_alert_spark_schema()
hit_schema = alert_hit_dict_factory.get_hit_spark_schema()

# Implementation: Spark manager
schema = StructType([
    StructField('alert_header', alert_schema),
    StructField('hits', ArrayType(hit_schema))
])

# file_name = 'RCMDB.ALERTS_SAMPLE.delta'
# Implementation: Spark manager
file_name = 'ALERTS.delta'
std_alert_df = spark_instance.read_delta("data/2.standardized/" + (file_name))

# Implementation: Spark manager
spark_instance.show_dim(std_alert_df)

# Implementation: Spark manager, XML Parser

alert_df = std_alert_df.withColumn('alert_hits',
                                   F.udf(alert_hit_extractor.extract_alert_hits_from_xml, schema)('html_file_key'))

%%time
# Implementation: Delta Converter

alert_df = spark_instance.write_and_get_delta_data(alert_df, in_cleansed_data_dir(file_name))

# Implementation: Spark manager
group_count(alert_df, 'alert_hits', 3)

# Implementation: Spark manager
alert_df.count()

# Implementation: Spark manager, XML Parser, Match/Hit Handler
alert_hits_df = alert_df.selectExpr('*', 'alert_hits.*') \
    .selectExpr('*', 'explode(hits) as hit') \
    .selectExpr('*', 'alert_header.*') \
    .selectExpr('*', 'hit.*') \
    .drop('alert_hits', 'alert_header', 'hits', 'hit')

%%time
# Implementation: Delta Exporter
alert_hits_df = write_read_delta(spark, alert_hits_df,
                                 in_cleansed_data_dir(file_name),
                                 user_metadata='At hit level'
                                )

# Implementation: XML Parser, Match/Hit Handler
alert_hits_df.select('alert_internal_id').distinct().count()

# Implementation: XML Parser, Match/Hit Handler
def get_wl_hit_aliases_matched_name(hit_aliases_displayName, hit_aliases_matchedName, hit_inputExplanations):
    if hit_inputExplanations is None or len(hit_inputExplanations) == 0:
        return []
    else:
        result = []
        hit_inputExplanations = list(set(hit_inputExplanations))
        for hit_inputExplanation in hit_inputExplanations:
            if hit_inputExplanation in hit_aliases_matchedName:
                index_in_matchedName = hit_aliases_matchedName.index(hit_inputExplanation)
                result.append(hit_aliases_displayName[index_in_matchedName])
            elif hit_inputExplanation in hit_aliases_displayName:
                result.append(hit_inputExplanation)
            else:
                result.append(hit_inputExplanation)
        
        return result

alert_hits_df.toPandas()["hit_inputExplanations_matchedName_inputExplanation"]



# Implementation: Match/Hit Handler, Spark manager
ap_hit_names_sql = sql_merge_to_target_col_from_source_cols(alert_hits_df, 'ap_hit_names', 'hit_inputExplanations_matchedName_inputExplanation', 'hit_inputExplanations_aliases_matchedName_inputExplanation')
alert_ap_hit_names_df = alert_hits_df.select('*', ap_hit_names_sql)

alert_ap_wl_hit_names_df = alert_ap_hit_names_df.withColumn('wl_hit_matched_name',
                                                              F.when(F.expr('size(hit_explanations_matchedName_Explanation) > 0'), F.col('hit_displayName')) \
                                                               .otherwise(F.lit(None))
                                                             ) \
                                                  .withColumn('wl_hit_aliases_matched_name',
                                                              udf(get_wl_hit_aliases_matched_name, ArrayType(StringType()))('hit_aliases_displayName', 'hit_aliases_matchedName', 'hit_explanations_aliases_matchedName_Explanation')
                                                             )

wl_hit_names_sql = sql_merge_to_target_col_from_source_cols(alert_ap_wl_hit_names_df, 'wl_hit_names', 'wl_hit_matched_name', 'wl_hit_aliases_matched_name')
alert_ap_wl_hit_names_df = alert_ap_wl_hit_names_df.select('*', wl_hit_names_sql)

alert_ap_wl_hit_names_df
len(alert_ap_wl_hit_names_df.toPandas()["wl_hit_aliases_matched_name"])

# Implementation: Match/Hit Handler, Spark manager
merge_hit_and_aliases_displayName_sql = sql_merge_to_target_col_from_source_cols(alert_ap_wl_hit_names_df, 'wl_hit_names', 'hit_displayName', 'hit_aliases_displayName')
merge_ap_names_sql = sql_merge_to_target_col_from_source_cols(alert_ap_wl_hit_names_df, 'ap_hit_names', 'alert_ahData_partyName', return_array=True)

alert_ap_wl_hit_names_df = alert_ap_wl_hit_names_df.withColumn('wl_hit_names',
                                                                 F.when(F.expr('size(wl_hit_names) > 0'), F.col('wl_hit_names')) \
                                                                  .otherwise(merge_hit_and_aliases_displayName_sql)
                                                                ) \
                                                   .withColumn('ap_hit_names',
                                                                 F.when(F.expr('size(ap_hit_names) > 0'), F.col('ap_hit_names')) \
                                                                  .otherwise(merge_ap_names_sql)
                                                                )

alert_ap_wl_hit_names_df.toPandas()['ap_hit_names']

In [30]:
# Implementation: Match/Hit Handler, Spark manager
alert_statuses_df = spark.read.format('delta').load(in_standardized_data_dir('ACM_MD_ALERT_STATUSES.delta')).select('STATUS_INTERNAL_ID', 'STATUS_NAME')
alert_ap_wl_hit_names_df = alert_ap_wl_hit_names_df.join(alert_statuses_df, 'STATUS_INTERNAL_ID')
alert_ap_wl_hit_names_df = reorder_cols(alert_ap_wl_hit_names_df, 'STATUS_INTERNAL_ID', 'STATUS_NAME')

In [31]:
%%time
# Implementation: Delta Exporter
alert_ap_wl_hit_names_df = write_read_delta(spark, alert_ap_wl_hit_names_df, in_cleansed_data_dir(file_name),
                                            user_metadata = 'More processing on AP and WL names, pinpoint to the exact names from AP and WL which caused the hits'
                                           )

CPU times: user 87.3 ms, sys: 2.3 ms, total: 89.6 ms
Wall time: 1.38 s


#### Below is not crucial?

In [32]:
# Implementation: NRIC handler (Customer specifics)

def extract_wl_nric_dob(custom_field):
    def _extract_yob_from_st_nric(nric):
        nric_type = nric[0]
        two_digit_year = nric[1:3]

        if nric_type.lower() == 's':
            if int(two_digit_year) >= 68:
                yob = '19' + two_digit_year
            else:
                yob = None
        else:
            yob = '20' + two_digit_year
            
        return yob
    
    nric_match = re.match('^NRIC:.*?([STGF]\d{7}[A-Z])', custom_field)
    dob_match = re.match('.*DOB: (.+?\d{4})[,.]', custom_field)
    possible_nric_match = re.findall('([STGF]\d{7}[A-Z])', custom_field)
    
    if nric_match:
        nric = nric_match.groups()
    else:
        nric = None
        
    if dob_match:
        dob = dob_match.groups()
    else:
        if nric and nric[0].lower() in ['s', 't']:
            dob = _extract_yob_from_st_nric(nric)
        else:
            dob = None
            
    if possible_nric_match:
        possible_nric = possible_nric_match
    else:
        possible_nric = None 
    
    return {'nric': nric, 'dob': dob, 'possible_nric': possible_nric}

extract_wl_nric_dob('NRIC: S6959726J, DOB: 1955, Freque')

alert_nric_df = alert_ap_wl_hit_names_df.withColumn('hit_cs_1_data_points',
                                                    udf(extract_wl_nric_dob, MapType(StringType(), ArrayType(StringType())))('hit_cs_1')
                                                   )

def extract_ap_nric(ap_id_numbers):
    ap_nrics = []
    for id_number in set(ap_id_numbers):
        if id_number and re.match('^[STGF]\d{7}[A-Z]$', id_number.upper()):
            ap_nrics.append(id_number)
    
    return ap_nrics

extract_ap_nric(['S7364776B', 'S7335736B'])

alert_nric_df = alert_nric_df.withColumn('ap_nric',
                                         udf(extract_ap_nric, ArrayType(StringType()))('alert_partyIds_idNumber')
                                        )

group_count(alert_nric_df.selectExpr('size(ap_nric) as s', 'ap_nric'), 's')

alert_nric_df.selectExpr('size(ap_nric) as s', 'ap_nric').where('s = 2').limit(2).toPandas()



21/12/09 11:22:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/09 11:22:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/09 11:22:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,s,ap_nric


In [33]:
%%time
alert_nric_df = write_read_delta(spark,
                                 alert_nric_df,
                                 in_cleansed_data_dir(file_name),
                                 user_metadata = 'Extracted AP and WL NRIC'
                                 )

CPU times: user 42.9 ms, sys: 2.55 ms, total: 45.4 ms
Wall time: 1 s


### Process alert notes and statuses


In [34]:
# Implementation: Spark manager
alert_notes_file_name = 'ACM_ALERT_NOTES.delta'
item_status_file_name = 'ACM_ITEM_STATUS_HISTORY.delta'

alert_notes_df = spark.read.format('delta').load(in_standardized_data_dir(alert_notes_file_name))
item_status_history_df = spark.read.format('delta').load(in_standardized_data_dir(item_status_file_name))
alert_statuses_df = spark.read.format('delta').load(in_standardized_data_dir('ACM_MD_ALERT_STATUSES.delta'))

Merge the "STATUS_NAME" to make the data more descriptive.

In [35]:
item_status_history_df.toPandas()

,STATUS_JOIN_ID,ITEM_JOIN_ID,ITEM_ID,FROM_STATUS_IDENTIFIER,FROM_STATE,FROM_FINDING,TO_STATUS_IDENTIFIER,TO_STATE,TO_FINDING,CREATE_DATE,USER_JOIN_ID
0,6059387,3658498,WLF101-1363601-89626,2,Open,No_Determination,3,Open,No_Determination,2021-08-16 18:02:47,34001
1,6059825,3658498,WLF101-1363601-89626,3,Open,No_Determination,IMPL_AML_FALSE_POSITIVE,Closed,Non_Issue,2021-08-17 09:03:03,30203
2,5780701,3541863,WLF101-939701-62908,None,None,None,2,Open,No_Determination,2021-03-17 14:52:06,22601
3,5135431,3292184,WLF101-945401-62939,2,Open,No_Determination,24,Open,No_Determination,2020-03-17 15:22:11,30202
4,5782825,3541863,WLF101-939701-62908,3,Open,No_Determination,IMPL_AML_FALSE_POSITIVE,Closed,Non_Issue,2021-03-18 12:04:25,30204
5,6059345,3658498,WLF101-1363601-89626,None,None,None,2,Open,No_Determination,2021-08-16 17:54:06,22601
6,5781004,3541863,WLF101-939701-62908,2,Open,No_Determination,3,Open,No_Determination,2021-03-17 15:00:55,34001
7,5800865,3292184,WLF101-945401-62939,24,Open,No_Determination,IMPL_AML_FALSE_POSITIVE,Closed,Non_Issue,2021-03-25 16:35:50,18506
8,5074477,3292184,WLF101-945401-62939,None,None,None,2,Open,No_Determination,2020-02-11 16:00:01,22601


In [36]:
# Implementation: Spark manager, Delta Converter

item_status_history_df.createOrReplaceTempView('status_df')

system_id = "22601"
item_status_history_stage_df = spark.sql(f'''
    with status_row_num as (
        select *,
            row_number() over (partition by item_id order by create_date asc) as row_num
        from status_df),
    first_last_analyst_row_num as (
        select ITEM_ID,
            min(row_num) as first_analyst_row_num,
            max(row_num) as last_analyst_row_num
        from status_row_num
        where user_join_id != "{system_id}"
        group by ITEM_ID
        )
    select a.*,
        b.first_analyst_row_num,
        b.last_analyst_row_num,
        case 
            when row_num = first_analyst_row_num and row_num = last_analyst_row_num then "first_last_analyst_status"
            when row_num = first_analyst_row_num then "first_analyst_status"
            when row_num = last_analyst_row_num then "last_analyst_status"
            when row_num > first_analyst_row_num then "middle_analyst_status"
            else "system_activity"
        end as analyst_status_stage
    from status_row_num a
    join first_last_analyst_row_num b
    on a.ITEM_ID = b.ITEM_ID
''')

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

In [37]:
%%time
a = write_read_delta(spark,
                                                item_status_history_stage_df,
                                                delta_path=in_cleansed_data_dir(item_status_file_name),
                                                user_metadata='Tagged the status stage'
                                               )

alert_notes_df.createOrReplaceTempView('notes_df')

alert_notes_stage_df = spark.sql('''
    with notes_row_num as (
        select *,
            row_number() over (partition by alert_id order by create_date asc) as row_num
        from notes_df),
    first_last_analyst_row_num as (
        select *,
            min(row_num) over (partition by alert_id) as first_analyst_row_num,
            max(row_num) over (partition by alert_id) as last_analyst_row_num
        from notes_row_num)
    select *,
        case 
            when row_num = first_analyst_row_num and row_num = last_analyst_row_num then "first_last_analyst_note"
            when row_num = first_analyst_row_num then "first_analyst_note"
            when row_num = last_analyst_row_num then "last_analyst_note"
            else "middle_analyst_note"
        end as analyst_note_stage
    from first_last_analyst_row_num    
''')

CPU times: user 2.43 ms, sys: 2.27 ms, total: 4.7 ms
Wall time: 1.7 s


In [38]:
%%time
alert_notes_stage_df = write_read_delta(spark,
                                        alert_notes_stage_df,
                                        delta_path=in_cleansed_data_dir(alert_notes_file_name),
                                        user_metadata='Tagged the note stage'
                                       )

extract_wl_nric_dob('NRIC: S6959726J, DOB: 1955, Freque')

alert_nric_df = alert_ap_wl_hit_names_df.withColumn('hit_cs_1_data_points',
                                                    udf(extract_wl_nric_dob, MapType(StringType(), ArrayType(StringType())))('hit_cs_1')
                                                   )

def extract_ap_nric(ap_id_numbers):
    ap_nrics = []
    for id_number in set(ap_id_numbers):
        if id_number and re.match('^[STGF]\d{7}[A-Z]$', id_number.upper()):
            ap_nrics.append(id_number)
    
    return ap_nrics

extract_ap_nric(['S7364776B', 'S7335736B'])

alert_nric_df = alert_nric_df.withColumn('ap_nric',
                                         udf(extract_ap_nric, ArrayType(StringType()))('alert_partyIds_idNumber')
                                        )

group_count(alert_nric_df.selectExpr('size(ap_nric) as s', 'ap_nric'), 's')

alert_nric_df.selectExpr('size(ap_nric) as s', 'ap_nric').where('s = 2').limit(2).toPandas()

21/12/09 11:22:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/09 11:22:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/09 11:22:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


CPU times: user 21.1 ms, sys: 9.02 ms, total: 30.1 ms
Wall time: 1.74 s


,s,ap_nric


In [39]:
%%time
alert_nric_df = write_read_delta(spark,
                                 alert_nric_df,
                                 in_cleansed_data_dir(file_name),
                                 user_metadata = 'Extracted AP and WL NRIC'
                                 )

CPU times: user 45.3 ms, sys: 354 µs, total: 45.6 ms
Wall time: 992 ms


### Process alert notes and statuses

In [40]:
# Spark manager
alert_notes_file_name = 'ACM_ALERT_NOTES.delta'
item_status_file_name = 'ACM_ITEM_STATUS_HISTORY.delta'

alert_notes_df = spark.read.format('delta').load(in_standardized_data_dir(alert_notes_file_name))
item_status_history_df = spark.read.format('delta').load(in_standardized_data_dir(item_status_file_name))
alert_statuses_df = spark.read.format('delta').load(in_standardized_data_dir('ACM_MD_ALERT_STATUSES.delta'))

Merge the "STATUS_NAME" to make the data more descriptive.

In [41]:
# Status preprocessor

item_status_history_df.createOrReplaceTempView('status_df')

system_id = "22601"
item_status_history_stage_df = spark.sql(f'''
    with status_row_num as (
        select *,
            row_number() over (partition by item_id order by create_date asc) as row_num
        from status_df),
    first_last_analyst_row_num as (
        select ITEM_ID,
            min(row_num) as first_analyst_row_num,
            max(row_num) as last_analyst_row_num
        from status_row_num
        where user_join_id != "{system_id}"
        group by ITEM_ID
        )
    select a.*,
        b.first_analyst_row_num,
        b.last_analyst_row_num,
        case 
            when row_num = first_analyst_row_num and row_num = last_analyst_row_num then "first_last_analyst_status"
            when row_num = first_analyst_row_num then "first_analyst_status"
            when row_num = last_analyst_row_num then "last_analyst_status"
            when row_num > first_analyst_row_num then "middle_analyst_status"
            else "system_activity"
        end as analyst_status_stage
    from status_row_num a
    join first_last_analyst_row_num b
    on a.ITEM_ID = b.ITEM_ID
''')

In [42]:
%%time
# Spark manager
item_status_history_stage_df = write_read_delta(spark,
                                                item_status_history_stage_df,
                                                delta_path=in_cleansed_data_dir(item_status_file_name),
                                                user_metadata='Tagged the status stage'
                                               )

CPU times: user 1.13 ms, sys: 1.19 ms, total: 2.32 ms
Wall time: 1.04 s


In [43]:
# Notes preprocessor
alert_notes_df.createOrReplaceTempView('notes_df')

alert_notes_stage_df = spark.sql('''
    with notes_row_num as (
        select *,
            row_number() over (partition by alert_id order by create_date asc) as row_num
        from notes_df),
    first_last_analyst_row_num as (
        select *,
            min(row_num) over (partition by alert_id) as first_analyst_row_num,
            max(row_num) over (partition by alert_id) as last_analyst_row_num
        from notes_row_num)
    select *,
        case 
            when row_num = first_analyst_row_num and row_num = last_analyst_row_num then "first_last_analyst_note"
            when row_num = first_analyst_row_num then "first_analyst_note"
            when row_num = last_analyst_row_num then "last_analyst_note"
            else "middle_analyst_note"
        end as analyst_note_stage
    from first_last_analyst_row_num    
''')

In [44]:
%%time
# Spark manager
alert_notes_stage_df = write_read_delta(spark,
                                        alert_notes_stage_df,
                                        delta_path=in_cleansed_data_dir(alert_notes_file_name),
                                        user_metadata='Tagged the note stage'
                                       )

CPU times: user 2.05 ms, sys: 0 ns, total: 2.05 ms
Wall time: 877 ms


# Create agent inputs - 3.0-cleansed-to-application-agent-input Spark manager, Note Preprocessor, Status Preprocessor, Agent input creator


# Load data

In [45]:
# file_name = 'RCMDB.ALERTS_SAMPLE.delta'
# Spark manager

alert_file_name = 'ALERTS.delta'
cleansed_alert_df = spark.read.format('delta').load(in_cleansed_data_dir(alert_file_name))

note_file_name = 'ACM_ALERT_NOTES.delta'
cleansed_note_df = spark.read.format('delta').load(in_cleansed_data_dir(note_file_name))

In [46]:
# Note Preprocessor, Status Preprocessor

cleansed_note_df = cleansed_note_df.where('analyst_note_stage like "%last%"').selectExpr('ALERT_ID', 'note as last_note')
cleansed_alert_df = cleansed_alert_df.join(cleansed_note_df, 'ALERT_ID', how='left')

In [47]:
group_count(cleansed_alert_df, 'last_note', 5)

21/12/09 11:22:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/09 11:22:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/09 11:22:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,last_note,count,percent,count_cum_sum,percent_cum_sum
0,<p>*1-3: Name mismatch</p>,1,33.333,1,33.333
1,"<p><span style=""font-family: 'courier new', courier;"">Number of hits : 1 </span></p>\n<p><span style=""font-family: 'courier new', courier;"">Party Name : ...",1,33.333,2,66.667
2,<p>RBA applied. Approval from JM to close the alerts.</p>,1,33.333,3,100.000


In [48]:
cleansed_alert_df.toPandas()

,ALERT_ID,STATUS_INTERNAL_ID,STATUS_NAME,ALERT_INTERNAL_ID,ENTITY_TYPE_ID,ALERT_DATE,ALERT_TYPE_ID,STATUS_ID,DELETED,HTML_FILE_KEY,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38,P39,P40,P41,P42,P43,P44,P45,P46,P47,P48,P49,IS_CASE,BUNIT_IDENTIFIER,OWNER_INTERNAL_ID,BU_INTERNAL_ID,ORIGINAL_BU_INTERNAL_ID,FL_ARCHIVE,FL_READ,FL_READ_BY_OWNER,LAST_READ_DATE,LAST_READ_USER_ID,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,CLOSED_DATE,CREATE_DATE,ALERT_CUSTOM_ATTRIBUTES_ID,SCORE,ALERT_TYPE_VERSION,FL_MANUAL,FL_GENERATED_BY_ACM,RESOLUTION_ID,ALERT_TYPE_INTERNAL_ID,FL_HAS_ATTACHMENTS,FL_UPDATED_BY_ACM,ENTITY_ID,PREV_STATUS_INTERNAL_ID,FL_ENCRYPTED,LAST_REFRESH_MODIFED_DATE,DEADLINE_DATE,HIGHLIGHT_DATE,EMAIL_DATE,AUTO_ESC_STATUS_INTERNAL_ID,CASE_COUNT_FOR_CONFIDENTIAL,P50,GLOBAL_DEADLINE_DATE,GLOBAL_HIGHLIGHT_DATE,GLOBAL_EMAIL_DATE,GLOBAL_AUTO_ESC_STATUS_ID,RFI_STATE,FL_HAS_NOTES,FL_HAS_CONFIDENTIAL_NOTES,CONSOLIDATION_KEY,HIBERNATE_OBJECT_VERSION,OWNER_IDENTIFIER,FL_DOUBT,NUM_EXISTING_ENTITIES,WORKSPACE_INTERNAL_ID,ALERT_NAME,PRIORITY_INTERNAL_ID,DETAILS_FOR_SEARCH,DETAILS,alert_alertId,alert_alertDate,alert_alertEntityKey,alert_score,alert_ahData_alertID,alert_ahData_alertDateTime,alert_ahData_jobID,alert_ahData_jobName,alert_ahData_jobType,alert_ahData_score,alert_ahData_numberOfHits,alert_ahData_partyKey,alert_ahData_partyName,alert_ahData_entityExcludeListsNames,alert_ahData_hitExcludeListsNames,alert_partyType,alert_partyDOB,alert_partyYOB,alert_partyBirthCountry,alert_partyBirthLocation,alert_partyGender,alert_partyIds_idType,alert_partyIds_idNumber,alert_partyIds_idCountry,alert_partyNatCountries_countryCd,alert_partyAddresses_partyAddressLine1,alert_partyAddresses_partyAddressLine2,alert_partyAddresses_partyCity,alert_partyAddresses_partyPostalCd,alert_partyAddresses_partyStateProvince,alert_partyAddresses_partyCountry,hit_listId,hit_entryId,hit_listVersion,hit_entryType,hit_listUpdateDate,hit_entryCreatedDate,hit_entryUpdateDate,hit_displayName,hit_matchedName,hit_isNameBroken,hit_aliases_displayName,hit_aliases_matchedName,hit_aliases_isNameBroken,hit_aliases_matchStrength,hit_addresses_streetAddress1,hit_addresses_streetAddress2,hit_addresses_city,hit_addresses_stateProvince,hit_addresses_postalCode,hit_addresses_country,hit_ids_idType,hit_ids_idNumber,hit_ids_idCountry,hit_nationalityCountries_country,hit_placesOfBirth_birthPlace,hit_placesOfBirth_birthCountry,hit_age,hit_ageAsOfDate,hit_datesOfBirth_birthDate,hit_datesOfBirth_yearOfBirth,hit_categories_category,hit_keywords_keyword,hit_title,hit_position,hit_gender,hit_isDeceased,hit_deceasedDate,hit_cs_1,hit_cs_2,hit_cs_3,hit_cs_4,hit_cs_5,hit_cs_6,hit_cs_7,hit_cs_8,hit_cs_9,hit_cs_10,hit_cs_11,hit_cs_12,hit_cs_13,hit_cs_14,hit_cs_15,hit_cs_16,hit_cs_17,hit_cs_18,hit_additionalInfo_name,hit_additionalInfo_value,hit_score,hit_matchType,hit_scoreFactors_factorId,hit_scoreFactors_factorDesc,hit_scoreFactors_factorValue,hit_scoreFactors_factorScore,hit_scoreFactors_factorImpact,hit_scoresBreakdown_matchedName,hit_scoresBreakdown_aliases_matchedName,hit_scoresBreakdown_addresses_city,hit_scoresBreakdown_addresses_country,hit_scoresBreakdown_addresses_stateProvince,hit_scoresBreakdown_ids_idNumber,hit_explanations_matchedName_Explanation,hit_explanations_aliases_matchedName_Explanation,hit_explanations_nationalityCountries_country_Explanation,hit_explanations_address_city_Explanation,hit_explanations_address_country_Explanation,hit_explanations_addresses_stateProvince_Explanation,hit_explanations_ids_idNumber_Explanation,hit_inputExplanations_matchedName_inputExplanation,hit_inputExplanations_aliases_matchedName_inputExplanation,hit_inputExplanations_nationalityCountries_country_inputExplanation,hit_inputExplanations_address_city_inputExplanation,hit_inputExplanations_address_country_inputExplanation,hit_inputExplanations_addresses_stateProvince_inputExplanation,hit_inputExplanations_ids_idNumber_inputExplanation,ap_hit_nam

__Detailed implementation__

It's rather easy to implement if the goal is just to produce the dataframe for agent to consume. Some interim data need to be created to serve the purpose of analytics.

There are 2 main categories of transformations.
1. Interface/config transformation: Activities on the agent input config/interface.
1. Data transformation: Activities on the data based on the config/interface.

Steps
1. Create the agent input config.
    1. Interface/config transformation. Define agent input template. Each agent's input is a dictionary with 4 key-value pairs.
    ```
    {
        'ap': [],
        'ap_aliases': [],
        'wl': [] ,
        'wl_aliases': []
    }
    ```

        - `ap`: The primary value(s) of alerted party's specific attribute, e.g, name, it could be from one or multiple columns.
        - `ap_aliases`: The aliases of alerted party's specific attribute, it could be from one or multiple columns.
        - So on and so forth for `wl` and `wl_aliases`.

    1. Interface/config transformation. Define the list of agents. __Each agent's name must end with `_agent`.__
    ```
    agent_list = [
        'name_agent',
        'gender_agent'
    ]
    ```

    1. Interface/config transformation. Config the agent input by specifying which column(s) should be treated as the input of which agent's which party's primary or aliase value(s). 
    ```
    {
        'name_agent': {
            'ap': ['record_name', 'short_name'],
            'ap_aliases': ['alternate_name'],
            'wl': ['name_hit'],
            'wl_aliases': []
        },
        'gender_agent': {'ap': ['record_gender'],
                         'ap_aliases': [],
                         'wl': ['additional_infos_gender'],
                         'wl_aliases': []
                        },
    }
    ```
    Certain concepts need to be defined here.
        1. `level-1-key`: The name of each agent, it's `name_agent` and `gender_agent`.
        1. `level-1-value`: The value of each agent's config, it's a dictionary, e.g, 
        ```
        {
            'ap': ['record_name', 'short_name'],
            'ap_aliases': ['alternate_name'],
            'wl': ['name_hit'],
            'wl_aliases': []
        }
        ```
        1. `level-2-key`: The key of each agent config's value, or rather the key of `level-1-value`. It's `ap`, `ap_aliases`, `wl` and `wl_aliases`.
        1. `level-2-value`: The list of column names, e.g, `['record_name', 'short_name']`.

1. Create the interim agent input config and data. The interface is standardized from here onwards.  In reality, the data format can be more complex, e.g, national IDs we need to consider both type and document number.
    1. Interface/config transformation. Prepend `level-1-key` to `level-2-key` so that `level-2-key` can be used as new column names to host the interim data for analytics and/or debugging activites. Take `name_agent` for example.
    ```
    {
        'name_agent': {
            'name_agent_ap': ['record_name', 'short_name'],
            'name_agent_ap_aliases': ['alternate_name'],
            'name_agent_wl': ['name_hit'],
            'name_agent_wl_aliases': []
        }
    }
    ```
    1. Data transformation. Merge the values from `level-2-value` columns to `level-2-key` column. Below table will be the result.
    
| uuid | record_name | short_name | alternate_name | wl_primary_name | name_hit |   name_agent_ap   | name_agent_ap_aliases | name_agent_wl | name_agent_wl_aliases |
| ---- | :---------: | :--------: | :------------: | :-------------: | :------: | :---------------: | :-------------------: | :-----------: | :-------------------: |
| 1234 |  Jim Green  |    J.G.    |      Jim       |   James Greg    |   J.G    | [Jim Green, J.G.] |          Jim          |      J.G      |         None          |

1. Create the final agent input config and data based on the standardized interface.
    1. Interface/config transformation. Now we have a consistent schema to create the 1 list of alerted party values and 1 list of watchlist party values. We no longer need to worry about the customer specific schema, e.g, `record_name`, `short_name` and etcs. They have been standardized as `name_agent_ap`, `name_agent_ap_aliases` and etcs.
    ```
    {
        'name_agent': {'ap_all_names_aggregated': ['name_agent_ap', 'name_agent_ap_aliases'],
                       'wl_all_names_aggregated': ['name_agent_wl', 'name_agent_wl_aliases']
                      }
    }
    ```
    1. Data transformation. Merge the values from the primary and alias columns. Below table will be the result.
    
| uuid | ap_all_names_aggregated | wl_all_names_aggregated |
| ---- | :---------------------: | :---------------------: |
| 1234 | [Jim Green, J.G., Jim]  |          [J.G]          |

In [49]:
# Agent input creator

import copy
input_template = {'ap': [],
                  'ap_aliases': [],
                  'wl': [] ,
                  'wl_aliases': []
                 }

agent_list = [
    'party_type_agent',
    'name_agent',
    'dob_agent',
    'pob_agent',
    'gender_agent',
    'national_id_agent',
#     'passport_agent',
    'document_number_agent',
    'nationality_agent',
    'historical_decision_name_agent',
    'pep_payment_agent',
    'hit_is_san_agent',
    'hit_is_deceased_agent',
    'hit_has_dob_id_address_agent',
    'rba_agent'
]

agent_input_config = {}

for agent in agent_list:
    new_input = copy.deepcopy(input_template)
    agent_input_config[agent] = new_input
    
agent_input_config

{'party_type_agent': {'ap': [], 'ap_aliases': [], 'wl': [], 'wl_aliases': []},
 'name_agent': {'ap': [], 'ap_aliases': [], 'wl': [], 'wl_aliases': []},
 'dob_agent': {'ap': [], 'ap_aliases': [], 'wl': [], 'wl_aliases': []},
 'pob_agent': {'ap': [], 'ap_aliases': [], 'wl': [], 'wl_aliases': []},
 'gender_agent': {'ap': [], 'ap_aliases': [], 'wl': [], 'wl_aliases': []},
 'national_id_agent': {'ap': [], 'ap_aliases': [], 'wl': [], 'wl_aliases': []},
 'document_number_agent': {'ap': [],
  'ap_aliases': [],
  'wl': [],
  'wl_aliases': []},
 'nationality_agent': {'ap': [], 'ap_aliases': [], 'wl': [], 'wl_aliases': []},
 'historical_decision_name_agent': {'ap': [],
  'ap_aliases': [],
  'wl': [],
  'wl_aliases': []},
 'pep_payment_agent': {'ap': [], 'ap_aliases': [], 'wl': [], 'wl_aliases': []},
 'hit_is_san_agent': {'ap': [], 'ap_aliases': [], 'wl': [], 'wl_aliases': []},
 'hit_is_deceased_agent': {'ap': [],
  'ap_aliases': [],
  'wl': [],
  'wl_aliases': []},
 'hit_has_dob_id_address_agent'

In [50]:
# Agent input creator

# DO NOTE the fff_format use i-based index While, python list is 0-based index
agent_input_config['party_type_agent']['ap'].extend(['alert_partyType'])
agent_input_config['party_type_agent']['wl'].extend(['hit_entryType'])

agent_input_config['name_agent']['ap'].extend(['ap_hit_names'])
agent_input_config['name_agent']['wl'].extend(['wl_hit_names'])

# agent_input_config['dob_agent']['ap'].extend(['alert_partyDOB', 'alert_partyYOB'])
# agent_input_config['dob_agent']['wl'].extend(['hit_datesOfBirth_birthDate', 'hit_datesOfBirth_yearOfBirth'])

# The alert_partyDOB has value of '31/12/99' which is actually '9999-12-31' from P14
agent_input_config['dob_agent']['ap'].extend(['P14'])
agent_input_config['dob_agent']['wl'].extend(['hit_datesOfBirth_birthDate', 'hit_cs_1_data_points.dob'])

agent_input_config['pob_agent']['ap'].extend(['alert_partyBirthCountry'])
# It's weird hit_placesOfBirth_birthPlace has country data instead of hit_placesOfBirth_birthCountry
agent_input_config['pob_agent']['wl'].extend(['hit_placesOfBirth_birthPlace'])

agent_input_config['gender_agent']['ap'].extend(['alert_partyGender'])
agent_input_config['gender_agent']['wl'].extend(['hit_gender'])

# The national ID is for SG NRIC only
agent_input_config['national_id_agent']['ap'].extend(['ap_nric'])
agent_input_config['national_id_agent']['wl'].extend(['hit_cs_1_data_points.nric'])

# agent_input_config['passport_agent']['ap'].extend(get_ap_screenable_attributes('PASSPORT'))
# agent_input_config['passport_agent']['wl'].extend(['passport'])
agent_input_config['document_number_agent']['ap'].extend(['alert_partyIds_idNumber'])
# The 'hit_cs_1_data_points.nric' is a subset of 'hit_cs_1_data_points.possible_nric'. Hence, use 'hit_cs_1_data_points.possible_nric' only
agent_input_config['document_number_agent']['wl'].extend(['hit_ids_idNumber', 'hit_cs_1_data_points.possible_nric'])

# "alert_partyNatCountries_countryCd" is alwasy empty, "alert_partyNatCountries_countryCd" is actually part of address
# but it's used by screening engine and analyst to match with WL nationality country
agent_input_config['nationality_agent']['ap'].extend(['alert_partyNatCountries_countryCd', 'alert_partyAddresses_partyCountry'])
agent_input_config['nationality_agent']['wl'].append('hit_nationalityCountries_country')

agent_input_config['historical_decision_name_agent']['ap'].extend(['alert_ahData_numberOfHits', 'STATUS_NAME', 'alert_ahData_partyName', 'ALERT_DATE'])
agent_input_config['historical_decision_name_agent']['wl'].extend(['hit_entryId'])

# agent_input_config['historical_decision_entity_key_agent']['ap'].extend(['alert_ahData_numberOfHits', 'STATUS_NAME', 'alert_alertEntityKey', 'ALERT_DATE'])
# agent_input_config['historical_decision_entity_key_agent']['wl'].extend(['hit_entryId'])

agent_input_config['pep_payment_agent']['ap'].extend(['P12'])
agent_input_config['pep_payment_agent']['wl'].extend(['hit_listID'])

# The P36 and P38 are at alert level, use `hit_listId` and `hit_categories_category` which is at hit level
agent_input_config['hit_is_san_agent']['wl'].extend(['hit_listId', 'hit_categories_category'])

agent_input_config['hit_is_deceased_agent']['wl'].extend(['hit_isDeceased'])

agent_input_config['hit_has_dob_id_address_agent']['wl'].extend(['hit_datesOfBirth_birthDate', 'hit_cs_1_data_points.dob', 'hit_ids_idNumber', 'hit_cs_1_data_points.possible_nric', 'hit_addresses_streetAddress1'])

agent_input_config['rba_agent']['ap'].extend(['alert_ahData_numberOfHits', 'STATUS_NAME', 'alert_ahData_partyName', 'ALERT_DATE', 'last_note'])
agent_input_config['rba_agent']['wl'].extend(['hit_entryId'])

In [51]:
# Agent input creator

def prepend_agent_name_to_ap_or_wl_or_aliases_key(agent_input_config):
    ''' Prepend the agent name (level 1 key) to level 2 key. So the new level 2 key will be 
    
        Input:
        { 'name_agent': {'ap': ['record_name'],
                         'ap_aliases': [],
                         'wl': ['name_hit'],
                         'wl_aliases': []
                        }
        }
        
        Output:
        { 'name_agent': {'name_agent_ap': ['record_name'],
                         'name_agent_ap_aliases': [],
                         'name_agent_wl': ['name_hit'],
                         'name_agent_wl_aliases': []
                        }
        }
    '''
    result = dict()
    for agent_name, config in agent_input_config.items():
        result[agent_name] = dict()
        
        for ap_or_wl_or_aliases, source_cols in config.items():
            prepended_key_name = '_'.join([agent_name, ap_or_wl_or_aliases])
            result[agent_name][prepended_key_name] = source_cols
            
    return result

In [52]:
# Sanity check
agent_input_prepended_agent_name_config = prepend_agent_name_to_ap_or_wl_or_aliases_key(agent_input_config)
agent_input_prepended_agent_name_config

{'party_type_agent': {'party_type_agent_ap': ['alert_partyType'],
  'party_type_agent_ap_aliases': [],
  'party_type_agent_wl': ['hit_entryType'],
  'party_type_agent_wl_aliases': []},
 'name_agent': {'name_agent_ap': ['ap_hit_names'],
  'name_agent_ap_aliases': [],
  'name_agent_wl': ['wl_hit_names'],
  'name_agent_wl_aliases': []},
 'dob_agent': {'dob_agent_ap': ['P14'],
  'dob_agent_ap_aliases': [],
  'dob_agent_wl': ['hit_datesOfBirth_birthDate', 'hit_cs_1_data_points.dob'],
  'dob_agent_wl_aliases': []},
 'pob_agent': {'pob_agent_ap': ['alert_partyBirthCountry'],
  'pob_agent_ap_aliases': [],
  'pob_agent_wl': ['hit_placesOfBirth_birthPlace'],
  'pob_agent_wl_aliases': []},
 'gender_agent': {'gender_agent_ap': ['alert_partyGender'],
  'gender_agent_ap_aliases': [],
  'gender_agent_wl': ['hit_gender'],
  'gender_agent_wl_aliases': []},
 'national_id_agent': {'national_id_agent_ap': ['ap_nric'],
  'national_id_agent_ap_aliases': [],
  'national_id_agent_wl': ['hit_cs_1_data_points.n

In [53]:
x = cleansed_alert_df.toPandas()

In [54]:
# Spark manager / Agent input creator

def spark_sql_create_agent_primary_alias_input_cols(df, agent_input_prepended_agent_name_config):
    ''' Merge the customer specific columns into standardized agent primary and alias input columns.
    
        Input:
        { 'name_agent': {'name_agent_ap': ['record_name', 'whatever_other_name'],
                         'name_agent_ap_aliases': [],
                         'name_agent_wl': ['name_hit'],
                         'name_agent_wl_aliases': []
                        }
        }
        
        Output:
        Take {'name_agent_ap': ['record_name', 'whatever_other_name']} as an example. The 2 columns
        'record_name' and 'whatever_other_name' will be merged to create a new column named 'name_agent_ap'.
    '''
    sql_expr_list = []
    
    for agent_name, config in agent_input_prepended_agent_name_config.items():
        for target_col, source_cols in config.items():
            
            sql_expr = sql_merge_to_target_col_from_source_cols(df, target_col, *source_cols)
            if sql_expr is not None:
                sql_expr_list.append(sql_expr)
                
    return sql_expr_list

In [55]:
# Agent input creator
agent_input_raw_df = cleansed_alert_df.select('*',
                                      *spark_sql_create_agent_primary_alias_input_cols(cleansed_alert_df, agent_input_prepended_agent_name_config))

In [56]:
x = agent_input_raw_df.toPandas()

In [57]:
agent_input_refined_df = agent_input_raw_df

In [58]:
# Agent input creator

def create_agent_input_agg_col_config(agent_input_prepended_agent_name_config):
    ''' Create the source and target columns based on the standardized agent input config.
    
        Input:
        { 'name_agent': {'name_agent_ap': ['record_name', 'whatever_other_name'],
                         'name_agent_ap_aliases': [],
                         'name_agent_wl': ['name_hit'],
                         'name_agent_wl_aliases': []
                        }
        }
        
        Output:
        {'name_agent': {'ap_all_names_aggregated': ['name_agent_ap', 'name_agent_ap_aliases'],
                        'wl_all_names_aggregated': ['name_agent_wl', 'name_agent_wl_aliases']
                       }
        }
    '''
    def _generate_simple_plural(word):
        if word.lower().endswith('s'):
            return word.lower() + 'es'
        elif word.lower().endswith('y') \
                and word.lower()[-2:] not in ['ay', 'ey', 'iy', 'oy', 'uy']:
            return word.lower()[:-1] + 'ies'
        else:
            return word.lower() + 's'
        
    def _get_ap_or_wl_agg_source_cols(level_1_value, party):
        source_cols = []
        for col in level_1_value.keys():
            if col.endswith(f'_{party}') or col.endswith(f'_{party}_aliases'):
                source_cols.append(col)
                
        return source_cols
        
    agent_input_agg_col_config = dict()
    
    for agent_name, config in agent_input_prepended_agent_name_config.items():
        agent_type = agent_name.split('_agent', 1)[0]
        
        agent_ap_agg_col = f"""ap_all_{_generate_simple_plural(agent_type)}_aggregated"""
        agent_wl_agg_col = f"""wl_all_{_generate_simple_plural(agent_type)}_aggregated"""
        
        agent_ap_agg_source_cols = _get_ap_or_wl_agg_source_cols(config, 'ap')
        agent_wl_agg_source_cols = _get_ap_or_wl_agg_source_cols(config, 'wl')
                
        agent_input_agg_col_config[agent_name] = dict()
        agent_input_agg_col_config[agent_name][agent_ap_agg_col] = agent_ap_agg_source_cols
        agent_input_agg_col_config[agent_name][agent_wl_agg_col] = agent_wl_agg_source_cols
            
    return agent_input_agg_col_config

In [59]:
# DO NOTE the input to the function is the config object after prepending agent name.
agent_input_agg_col_config = create_agent_input_agg_col_config(agent_input_prepended_agent_name_config)
agent_input_agg_col_config

{'party_type_agent': {'ap_all_party_types_aggregated': ['party_type_agent_ap',
   'party_type_agent_ap_aliases'],
  'wl_all_party_types_aggregated': ['party_type_agent_wl',
   'party_type_agent_wl_aliases']},
 'name_agent': {'ap_all_names_aggregated': ['name_agent_ap',
   'name_agent_ap_aliases'],
  'wl_all_names_aggregated': ['name_agent_wl', 'name_agent_wl_aliases']},
 'dob_agent': {'ap_all_dobs_aggregated': ['dob_agent_ap',
   'dob_agent_ap_aliases'],
  'wl_all_dobs_aggregated': ['dob_agent_wl', 'dob_agent_wl_aliases']},
 'pob_agent': {'ap_all_pobs_aggregated': ['pob_agent_ap',
   'pob_agent_ap_aliases'],
  'wl_all_pobs_aggregated': ['pob_agent_wl', 'pob_agent_wl_aliases']},
 'gender_agent': {'ap_all_genders_aggregated': ['gender_agent_ap',
   'gender_agent_ap_aliases'],
  'wl_all_genders_aggregated': ['gender_agent_wl', 'gender_agent_wl_aliases']},
 'national_id_agent': {'ap_all_national_ids_aggregated': ['national_id_agent_ap',
   'national_id_agent_ap_aliases'],
  'wl_all_nationa

In [60]:
# Agent input creator

# The agent_agg_cols_config will be needed later
with open('agent_input_agg_col_config.json', 'w') as outfile:
    json.dump(agent_input_agg_col_config, outfile)

In [61]:
# Spark manager, Agent input creator

def spark_sql_create_agg_cols(df, agent_input_agg_col_config):
    sql_expr_list = []
    for agent, config in agent_input_agg_col_config.items():
        for party_agg_col, party_agg_source_cols in config.items():
            target_col = party_agg_col
            source_cols = party_agg_source_cols
            
            sql_expr = sql_merge_to_target_col_from_source_cols(df, target_col, *source_cols, return_array=True)
            if sql_expr is not None:
                sql_expr_list.append(sql_expr)

    return sql_expr_list

In [62]:
%%time
# Agent input creator
agent_input_agg_df = agent_input_refined_df.select('*',
                                                   *spark_sql_create_agg_cols(agent_input_refined_df, agent_input_agg_col_config)
                                                  ) \
                                            .withColumn('_index', F.monotonically_increasing_id())

agent_input_agg_df = write_read_delta(spark,
                                     agent_input_agg_df,
                                     in_application_data_dir('agent_input_agg_df.delta')
                                    )

CPU times: user 196 ms, sys: 22.5 ms, total: 218 ms
Wall time: 2.24 s


In [63]:
agent_input_agg_df.toPandas().head()

,ALERT_ID,STATUS_INTERNAL_ID,STATUS_NAME,ALERT_INTERNAL_ID,ENTITY_TYPE_ID,ALERT_DATE,ALERT_TYPE_ID,STATUS_ID,DELETED,HTML_FILE_KEY,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38,P39,P40,P41,P42,P43,P44,P45,P46,P47,P48,P49,IS_CASE,BUNIT_IDENTIFIER,OWNER_INTERNAL_ID,BU_INTERNAL_ID,ORIGINAL_BU_INTERNAL_ID,FL_ARCHIVE,FL_READ,FL_READ_BY_OWNER,LAST_READ_DATE,LAST_READ_USER_ID,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,CLOSED_DATE,CREATE_DATE,ALERT_CUSTOM_ATTRIBUTES_ID,SCORE,ALERT_TYPE_VERSION,FL_MANUAL,FL_GENERATED_BY_ACM,RESOLUTION_ID,ALERT_TYPE_INTERNAL_ID,FL_HAS_ATTACHMENTS,FL_UPDATED_BY_ACM,ENTITY_ID,PREV_STATUS_INTERNAL_ID,FL_ENCRYPTED,LAST_REFRESH_MODIFED_DATE,DEADLINE_DATE,HIGHLIGHT_DATE,EMAIL_DATE,AUTO_ESC_STATUS_INTERNAL_ID,CASE_COUNT_FOR_CONFIDENTIAL,P50,GLOBAL_DEADLINE_DATE,GLOBAL_HIGHLIGHT_DATE,GLOBAL_EMAIL_DATE,GLOBAL_AUTO_ESC_STATUS_ID,RFI_STATE,FL_HAS_NOTES,FL_HAS_CONFIDENTIAL_NOTES,CONSOLIDATION_KEY,HIBERNATE_OBJECT_VERSION,OWNER_IDENTIFIER,FL_DOUBT,NUM_EXISTING_ENTITIES,WORKSPACE_INTERNAL_ID,ALERT_NAME,PRIORITY_INTERNAL_ID,DETAILS_FOR_SEARCH,DETAILS,alert_alertId,alert_alertDate,alert_alertEntityKey,alert_score,alert_ahData_alertID,alert_ahData_alertDateTime,alert_ahData_jobID,alert_ahData_jobName,alert_ahData_jobType,alert_ahData_score,alert_ahData_numberOfHits,alert_ahData_partyKey,alert_ahData_partyName,alert_ahData_entityExcludeListsNames,alert_ahData_hitExcludeListsNames,alert_partyType,alert_partyDOB,alert_partyYOB,alert_partyBirthCountry,alert_partyBirthLocation,alert_partyGender,alert_partyIds_idType,alert_partyIds_idNumber,alert_partyIds_idCountry,alert_partyNatCountries_countryCd,alert_partyAddresses_partyAddressLine1,alert_partyAddresses_partyAddressLine2,alert_partyAddresses_partyCity,alert_partyAddresses_partyPostalCd,alert_partyAddresses_partyStateProvince,alert_partyAddresses_partyCountry,hit_listId,hit_entryId,hit_listVersion,hit_entryType,hit_listUpdateDate,hit_entryCreatedDate,hit_entryUpdateDate,hit_displayName,hit_matchedName,hit_isNameBroken,hit_aliases_displayName,hit_aliases_matchedName,hit_aliases_isNameBroken,hit_aliases_matchStrength,hit_addresses_streetAddress1,hit_addresses_streetAddress2,hit_addresses_city,hit_addresses_stateProvince,hit_addresses_postalCode,hit_addresses_country,hit_ids_idType,hit_ids_idNumber,hit_ids_idCountry,hit_nationalityCountries_country,hit_placesOfBirth_birthPlace,hit_placesOfBirth_birthCountry,hit_age,hit_ageAsOfDate,hit_datesOfBirth_birthDate,hit_datesOfBirth_yearOfBirth,hit_categories_category,hit_keywords_keyword,hit_title,hit_position,hit_gender,hit_isDeceased,hit_deceasedDate,hit_cs_1,hit_cs_2,hit_cs_3,hit_cs_4,hit_cs_5,hit_cs_6,hit_cs_7,hit_cs_8,hit_cs_9,hit_cs_10,hit_cs_11,hit_cs_12,hit_cs_13,hit_cs_14,hit_cs_15,hit_cs_16,hit_cs_17,hit_cs_18,hit_additionalInfo_name,hit_additionalInfo_value,hit_score,hit_matchType,hit_scoreFactors_factorId,hit_scoreFactors_factorDesc,hit_scoreFactors_factorValue,hit_scoreFactors_factorScore,hit_scoreFactors_factorImpact,hit_scoresBreakdown_matchedName,hit_scoresBreakdown_aliases_matchedName,hit_scoresBreakdown_addresses_city,hit_scoresBreakdown_addresses_country,hit_scoresBreakdown_addresses_stateProvince,hit_scoresBreakdown_ids_idNumber,hit_explanations_matchedName_Explanation,hit_explanations_aliases_matchedName_Explanation,hit_explanations_nationalityCountries_country_Explanation,hit_explanations_address_city_Explanation,hit_explanations_address_country_Explanation,hit_explanations_addresses_stateProvince_Explanation,hit_explanations_ids_idNumber_Explanation,hit_inputExplanations_matchedName_inputExplanation,hit_inputExplanations_aliases_matchedName_inputExplanation,hit_inputExplanations_nationalityCountries_country_inputExplanation,hit_inputExplanations_address_city_inputExplanation,hit_inputExplanations_address_country_inputExplanation,hit_inputExplanations_addresses_stateProvince_inputExplanation,hit_inputExplanations_ids_idNumber_inputExplanation,ap_hit_nam

In [64]:
spark.read.format('delta').load(in_application_data_dir('agent_input_agg_df.delta')).count()

3

In [65]:
to_pandas(agent_input_agg_df)

,ALERT_ID,STATUS_INTERNAL_ID,STATUS_NAME,ALERT_INTERNAL_ID,ENTITY_TYPE_ID,ALERT_DATE,ALERT_TYPE_ID,STATUS_ID,DELETED,HTML_FILE_KEY,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,P38,P39,P40,P41,P42,P43,P44,P45,P46,P47,P48,P49,IS_CASE,BUNIT_IDENTIFIER,OWNER_INTERNAL_ID,BU_INTERNAL_ID,ORIGINAL_BU_INTERNAL_ID,FL_ARCHIVE,FL_READ,FL_READ_BY_OWNER,LAST_READ_DATE,LAST_READ_USER_ID,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,CLOSED_DATE,CREATE_DATE,ALERT_CUSTOM_ATTRIBUTES_ID,SCORE,ALERT_TYPE_VERSION,FL_MANUAL,FL_GENERATED_BY_ACM,RESOLUTION_ID,ALERT_TYPE_INTERNAL_ID,FL_HAS_ATTACHMENTS,FL_UPDATED_BY_ACM,ENTITY_ID,PREV_STATUS_INTERNAL_ID,FL_ENCRYPTED,LAST_REFRESH_MODIFED_DATE,DEADLINE_DATE,HIGHLIGHT_DATE,EMAIL_DATE,AUTO_ESC_STATUS_INTERNAL_ID,CASE_COUNT_FOR_CONFIDENTIAL,P50,GLOBAL_DEADLINE_DATE,GLOBAL_HIGHLIGHT_DATE,GLOBAL_EMAIL_DATE,GLOBAL_AUTO_ESC_STATUS_ID,RFI_STATE,FL_HAS_NOTES,FL_HAS_CONFIDENTIAL_NOTES,CONSOLIDATION_KEY,HIBERNATE_OBJECT_VERSION,OWNER_IDENTIFIER,FL_DOUBT,NUM_EXISTING_ENTITIES,WORKSPACE_INTERNAL_ID,ALERT_NAME,PRIORITY_INTERNAL_ID,DETAILS_FOR_SEARCH,DETAILS,alert_alertId,alert_alertDate,alert_alertEntityKey,alert_score,alert_ahData_alertID,alert_ahData_alertDateTime,alert_ahData_jobID,alert_ahData_jobName,alert_ahData_jobType,alert_ahData_score,alert_ahData_numberOfHits,alert_ahData_partyKey,alert_ahData_partyName,alert_ahData_entityExcludeListsNames,alert_ahData_hitExcludeListsNames,alert_partyType,alert_partyDOB,alert_partyYOB,alert_partyBirthCountry,alert_partyBirthLocation,alert_partyGender,alert_partyIds_idType,alert_partyIds_idNumber,alert_partyIds_idCountry,alert_partyNatCountries_countryCd,alert_partyAddresses_partyAddressLine1,alert_partyAddresses_partyAddressLine2,alert_partyAddresses_partyCity,alert_partyAddresses_partyPostalCd,alert_partyAddresses_partyStateProvince,alert_partyAddresses_partyCountry,hit_listId,hit_entryId,hit_listVersion,hit_entryType,hit_listUpdateDate,hit_entryCreatedDate,hit_entryUpdateDate,hit_displayName,hit_matchedName,hit_isNameBroken,hit_aliases_displayName,hit_aliases_matchedName,hit_aliases_isNameBroken,hit_aliases_matchStrength,hit_addresses_streetAddress1,hit_addresses_streetAddress2,hit_addresses_city,hit_addresses_stateProvince,hit_addresses_postalCode,hit_addresses_country,hit_ids_idType,hit_ids_idNumber,hit_ids_idCountry,hit_nationalityCountries_country,hit_placesOfBirth_birthPlace,hit_placesOfBirth_birthCountry,hit_age,hit_ageAsOfDate,hit_datesOfBirth_birthDate,hit_datesOfBirth_yearOfBirth,hit_categories_category,hit_keywords_keyword,hit_title,hit_position,hit_gender,hit_isDeceased,hit_deceasedDate,hit_cs_1,hit_cs_2,hit_cs_3,hit_cs_4,hit_cs_5,hit_cs_6,hit_cs_7,hit_cs_8,hit_cs_9,hit_cs_10,hit_cs_11,hit_cs_12,hit_cs_13,hit_cs_14,hit_cs_15,hit_cs_16,hit_cs_17,hit_cs_18,hit_additionalInfo_name,hit_additionalInfo_value,hit_score,hit_matchType,hit_scoreFactors_factorId,hit_scoreFactors_factorDesc,hit_scoreFactors_factorValue,hit_scoreFactors_factorScore,hit_scoreFactors_factorImpact,hit_scoresBreakdown_matchedName,hit_scoresBreakdown_aliases_matchedName,hit_scoresBreakdown_addresses_city,hit_scoresBreakdown_addresses_country,hit_scoresBreakdown_addresses_stateProvince,hit_scoresBreakdown_ids_idNumber,hit_explanations_matchedName_Explanation,hit_explanations_aliases_matchedName_Explanation,hit_explanations_nationalityCountries_country_Explanation,hit_explanations_address_city_Explanation,hit_explanations_address_country_Explanation,hit_explanations_addresses_stateProvince_Explanation,hit_explanations_ids_idNumber_Explanation,hit_inputExplanations_matchedName_inputExplanation,hit_inputExplanations_aliases_matchedName_inputExplanation,hit_inputExplanations_nationalityCountries_country_inputExplanation,hit_inputExplanations_address_city_inputExplanation,hit_inputExplanations_address_country_inputExplanation,hit_inputExplanations_addresses_stateProvince_inputExplanation,hit_inputExplanations_ids_idNumber_inputExplanation,ap_hit_nam

In [66]:
%%time
# Agent input creator
key_cols = ['_index', 'ALERT_INTERNAL_ID', 'ALERT_ID', 'hit_listId', 'hit_entryId']
for agent_name, input_agg_col_config in agent_input_agg_col_config.items():
    start = time.time()
    
    if agent_name in ['name_agent', 'dob_agent']:
        agent_input_df = agent_input_agg_df.select(*key_cols,
                                             *input_agg_col_config.keys(),
                                             'party_type_agent_ap',
                                             'party_type_agent_wl'
                                            )
    elif 'pary_type' in agent_name:
        continue
    else:
        agent_input_df = agent_input_agg_df.select(*key_cols, *input_agg_col_config.keys())
    
    # Our agent support the input list has None, hence, filter out None from all the agg columns (they will be the agent inputs)
    for agg_col_name in input_agg_col_config.keys():
        agent_input_df = agent_input_df.withColumn(agg_col_name, F.expr(f'array_except({agg_col_name}, array(null))'))
        
    agent_input_df_path = in_application_data_dir(f'agent-input/{agent_name}_input.delta')
    agent_input_df = write_read_delta(spark, agent_input_df, agent_input_df_path)
    logging.info(f'Agent: {agent}, Input written to {agent_input_df_path}, elapsed time: {time.time() - start:.2f}s')

2021/12/09 11:22:59 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/party_type_agent_input.delta, elapsed time: 1.03s
2021/12/09 11:23:00 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/name_agent_input.delta, elapsed time: 1.09s
2021/12/09 11:23:02 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/dob_agent_input.delta, elapsed time: 1.04s
2021/12/09 11:23:03 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/pob_agent_input.delta, elapsed time: 1.02s
2021/12/09 11:23:04 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/gender_agent_input.delta, elapsed time: 1.03s
2021/12/09 11:23:05 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/national_id_agent_input.delta, elapsed time: 1.04s
2021/12/09 11:23:06 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/document_number_

CPU times: user 58.9 ms, sys: 5.72 ms, total: 64.6 ms
Wall time: 14.3 s


# Test agent (optional if we have time) - 3.2.0 - Agent Manager, Spark Manager

In [67]:
# Agent Manager

from google.protobuf.json_format import MessageToJson

In [68]:
# Agent Manager

def get_agent_input_output_path(agent_name):
    def _get_agent_input_path(input_dir, agent_name):
        return os.path.join(input_dir, agent_name+'_input.delta')

    def _get_agent_output_path(output_dir, agent_name):
        os.makedirs(output_dir, exist_ok=True)
        return os.path.join(output_dir, agent_name+'_output.csv')
    
    agent_input_dir_name = in_application_data_dir('agent-input')
    agent_output_dir_name = agent_input_dir_name.replace('input', 'output')

    agent_input_path = _get_agent_input_path(agent_input_dir_name, agent_name)
    agent_output_path = _get_agent_output_path(agent_output_dir_name, agent_name)
    
    return agent_input_path, agent_output_path

In [69]:
# Agent Manager
agent_name = 'name_agent'

agent_input_path, agent_output_path = get_agent_input_output_path(agent_name)
print(agent_input_path)
print(agent_output_path)

./data/4.application/agent-input/name_agent_input.delta
./data/4.application/agent-output/name_agent_output.csv


In [70]:
# Agent Manager
agent_input_df = spark.read.format('delta').load(agent_input_path)

# Limit the name agent to 6 threads only, because other agents run on single thread, e.g, there are 4 doc
# and they need to run sequentially
agent_input_df = agent_input_df.repartition(16)

instance_name_mapping = {
'PERSON': 'advname-indv',
'ORGANIZATION': 'advname-org',
}

instance_name_col = 'name_agent_config'
alerted_names_col = 'ap_all_names_aggregated'
matched_names_col = 'wl_all_names_aggregated'

In [71]:
%%time
# Agent Manager
agent_input_df = agent_input_df.withColumn(instance_name_col, F.lit(''))

for key, value in instance_name_mapping.items():

    agent_input_df = agent_input_df.withColumn(
        instance_name_col,
        F.when(F.col('party_type_agent_wl') == key, value).otherwise(F.col(instance_name_col))
    )

display(agent_input_df.limit(10).toPandas())

,_index,ALERT_INTERNAL_ID,ALERT_ID,hit_listId,hit_entryId,ap_all_names_aggregated,wl_all_names_aggregated,party_type_agent_ap,party_type_agent_wl,name_agent_config
0,1,1619405,WLF101-939701-62908,FACTIVA_SAN,4790496,[P ONE],[One P],Person,PERSON,advname-indv
1,0,1649364,WLF101-1363601-89626,FACTIVA_SIE,1091285,[CPF BOARD],[CPF],Organization,ORGANIZATION,advname-org
2,2,1619436,WLF101-945401-62939,FACTIVA_SAN,1198704,[KIM],"[Kim,]",Person,PERSON,advname-indv


CPU times: user 14.9 ms, sys: 0 ns, total: 14.9 ms
Wall time: 135 ms


In [72]:
# Agent Manager
group_count(agent_input_df, 'name_agent_config')

21/12/09 11:23:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/09 11:23:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/09 11:23:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,name_agent_config,count,percent,count_cum_sum,percent_cum_sum
0,advname-indv,2,66.667,2,66.667
1,advname-org,1,33.333,3,100.000


In [73]:
print('test')

test


In [74]:
import utils.agent_service_standalone as standalone_agents

TypeError: Couldn't build proto file into descriptor pool!
Invalid proto descriptor for file "silenteight/agent/document/v1/api/document_numbers_comparer_agent.proto":
  silenteight.agent.document.v1.api.CompareDocumentNumbersRequest: "silenteight.agent.document.v1.api.CompareDocumentNumbersRequest" is already defined in file "silenteight/agent/document/v1/api/document_agent.proto".
  silenteight.agent.document.v1.api.CompareDocumentNumbersResponse: "silenteight.agent.document.v1.api.CompareDocumentNumbersResponse" is already defined in file "silenteight/agent/document/v1/api/document_agent.proto".
  silenteight.agent.document.v1.api.DocumentNumbersComparerAgent.CompareDocumentNumbers: "silenteight.agent.document.v1.api.CompareDocumentNumbersRequest" seems to be defined in "silenteight/agent/document/v1/api/document_agent.proto", which is not imported by "silenteight/agent/document/v1/api/document_numbers_comparer_agent.proto".  To use it here, please add the necessary import.
  silent

In [ ]:
# Agent Manager

def call_name_agent(instance_name, alerted_names, matched_names):
    def _remove_none_from_list(values):
        return [v for v in values if v is not None]
    
    def _get_reason_as_json(reason):
        if reason is None:
            return '{}'
        elif isinstance(reason, (dict, list)):
            return json.dumps(reason)
        else:
            return MessageToJson(reason)
        
#     response = agentservice.call_name_agent(instance_name, alerted_names, matched_names)
    alerted_names = _remove_none_from_list(alerted_names)
    matched_names = _remove_none_from_list(matched_names)
    response = standalone_agents.call_name_agent(instance_name, alerted_names, matched_names)

    return response.result, _get_reason_as_json(response.reason)

# Spark manager
name_agent_schema = StructType([
    StructField('name_agent', StringType()),
    StructField('name_agent_reason', StringType())
])

In [ ]:
# Agent Manager

call_name_agent('advname-org', ['cpf'], ['cpf board'])

In [ ]:
# Spark manager

agent_output_df = agent_input_df.withColumn('name_agent_all',
                                            udf(call_name_agent, name_agent_schema)(instance_name_col, alerted_names_col, matched_names_col)
                                           ) \
                                .select('*', 'name_agent_all.*') \
                                .drop('name_agent_all')

In [ ]:
# Agent Manager

agent_output_df.limit(10).toPandas()
